In [12]:
# Import libraries
import requests
import pandas as pd
import sqlite3
from functions import *

In [13]:
# Replace with your league ID
league_id = 1642687

# Base URL for the API
base_url = "https://fantasy.premierleague.com/api"

# The total number of gameweeks played
total_gameweeks = 33

# Define the database filename
database_file = 'pl_data.db'
create_database(database_file)

colours = [
    "#77AADD",
    "#EE8866",
    "#EEDD88",
    "#FFAABB",
    "#99DDFF",
    "#44BB99",
    "#BBCC33",
    "#AAAA00",
    "#DDDDDD",
    "#882255"
]

In [14]:
# Get league standings data
entries = {}
standings_data = get_data(base_url,f"leagues-classic/{league_id}/standings")
dfs = []
for item in standings_data["standings"]["results"]:
    dfs.append(pd.DataFrame(item,index=[0]))
team_data_df = pd.concat(dfs,ignore_index=True)
team_data_df["first_name"] = [x[0] for x in team_data_df.player_name.str.split(" ")]
team_data_df["last_name"] = [x[-1] for x in team_data_df.player_name.str.split(" ")]

df_to_db(team_data_df,"pl_data.db","player_data")
team_data_df

,id,event_total,player_name,rank,last_rank,rank_sort,total,entry,entry_name,first_name,last_name
0,61199436,66,Louis Holmes,1,1,1,1840,7331837,Lou Lou's Big Boys,Louis,Holmes
1,59942998,74,Oliver Chapman,2,2,2,1793,7204779,Geriatric FC,Oliver,Chapman
2,87205312,82,Harry Wilson,3,3,3,1777,5853213,Show me the Mané,Harry,Wilson
3,65638775,73,Sebastian H,4,4,4,1744,7759705,Übermenschen,Sebastian,H
4,63565261,52,Matt Johnson,5,5,5,1711,7551301,1 Pound Fishermen,Matt,Johnson
5,60163544,62,Joseph Barry-Shaw,6,6,6,1698,7228926,Last Place FC,Joseph,Barry-Shaw
6,62715982,63,Benjamin Kelly,7,7,7,1652,7481365,MusgysLoser,Benjamin,Kelly
7,60595555,58,Christopher Culpin,8,8,8,1637,7270610,We Cheat FR,Christopher,Culpin
8,63317631,55,Zak beresford,9,9,9,1618,7539556,RimmiesRus,Zak,beresford
9,60949807,47,Callum Waller,10,10,10,1537,7307161,HUMP,Callum,Waller


In [15]:
# Example usage:
colour_dict = {team_data_df.player_name.values[i]:colours[i] for i in range(len(team_data_df.player_name))}

In [16]:
team_datas = {}
gameweek_totals_df = pd.DataFrame()
for team_id in team_data_df.entry:
    url = f"https://fantasy.premierleague.com/api/entry/{team_id}/history/"
    response = requests.get(url)
    team_datas[team_id] = response.json()

In [17]:
gameweek_totals_df = get_timeseries(team_datas,"total_points")
df_to_db(gameweek_totals_df,"pl_data.db","gameweek_running_totals")

gameweek_std_totals_df = gameweek_totals_df.copy()
for col in gameweek_std_totals_df:
    gameweek_std_totals_df[col] = gameweek_std_totals_df[col]/gameweek_std_totals_df.index

gameweek_points_df = get_timeseries(team_datas,"points")
df_to_db(gameweek_points_df,"pl_data.db","gameweek_points")

In [18]:
plot_timeseries(gameweek_totals_df,team_data_df)

In [19]:
fig = plot_timeseries_bar_by_gw(gameweek_totals_df,colour_dict,team_data_df)
fig.update_layout(
    title="Total points per player",
    xaxis_title="Points",
    yaxis_title="Player name",
    legend_title="Players",
    font=dict(
        color="RebeccaPurple"
    )
)
fig.show()

In [20]:
fig = plot_timeseries_bar_by_gw(gameweek_points_df,colour_dict,team_data_df)
fig.update_layout(
    title="Gameweek points per player",
    xaxis_title="Points",
    yaxis_title="Player name",
    legend_title="Players",
    font=dict(
        color="RebeccaPurple"
    )
)
fig.show()

In [21]:
df = get_weeks_first_last(gameweek_totals_df)
fig = plot_vals_per_player(df,[("weeks_first","Weeks in first place"),("weeks_last","Weeks in last place")])
fig.update_layout(
    title="Weeks spent in first and last place per player",
    xaxis_title="Points",
    yaxis_title="Player name",
    legend_title="Case",
    barmode='stack',
    xaxis_tickangle=-45,
    font=dict(
        color="RebeccaPurple"
    )
)
fig.show()

In [22]:
fig = go.Figure()
players = [id_to_name(x,team_data_df) for x in df.entry]

fig = go.Figure(data=[
    go.Bar(name='Weeks at first', x=players, y=df.weeks_first,marker_color=colours[0]),
    go.Bar(name='Weeks at last', x=players, y=df.weeks_last,marker_color=colours[1])
])
fig.update_layout(
    title="Weeks spent at first and last place per player",
    xaxis_title="Points",
    yaxis_title="Player name",
    legend_title="Case",
    barmode='stack',
    xaxis_tickangle=-45,
    font=dict(
        color="RebeccaPurple"
    )
)
fig.show()